In [28]:
import os
import json
from collections import defaultdict
from tqdm import tqdm

def parse_cord_v2_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    grouped = defaultdict(lambda: {"count": "", "name": "", "price": ""})

    total_price = ""
    
    for line in data["valid_line"]:
        category = line["category"]
        group_id = line["group_id"]
        words = " ".join(w["text"] for w in line["words"])

        if category == "menu.cnt":
            grouped[group_id]["count"] = words
        elif category in ["menu.nm", "menu.sub_nm"]:
            grouped[group_id]["name"] += words + " "
        elif category == "menu.price":
            grouped[group_id]["price"] = words
        elif category == "total.total_price":
            total_price = words

    items = []
    for g in sorted(grouped.keys()):
        entry = grouped[g]
        name = entry["name"].strip()
        if name and entry["price"]:
            items.append({
                "name": name,
                "count": entry["count"],
                "price": entry["price"]
            })

    result = {
        "items": items,
        "total": total_price
    }

    return result

def prepare_donut_cord_dataset(json_dir, image_dir, output_path):
    dataset = []

    json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

    for json_file in tqdm(json_files, desc="Processing JSON files"):
        json_path = os.path.join(json_dir, json_file)
        image_name = os.path.splitext(json_file)[0] + ".png"  # nếu là .jpg sửa ở đây
        image_path = os.path.join(image_dir, image_name)

        if not os.path.exists(image_path):
            print(f"Image not found for {json_file}, skipping...")
            continue

        text_output = parse_cord_v2_json(json_path)
        
        dataset.append({
            "id": json_file.replace(".json", ""),
            "image_path": image_path,
            "text_input": "<s_cord-v2>",  # prefix task
            "text_output": json.dumps(text_output, ensure_ascii=False)
        })

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)

    print(f"✅ Done! Saved {len(dataset)} samples to {output_path}")

# Cách chạy
prepare_donut_cord_dataset(
    json_dir="data/json",
    image_dir="data/image",
    output_path="data/train.json"
)


Processing JSON files:  23%|██▎       | 165/733 [00:00<00:00, 1645.79it/s]

Processing JSON files: 100%|██████████| 733/733 [00:00<00:00, 1352.86it/s]


✅ Done! Saved 733 samples to data/train.json


In [50]:
import os

image_dir = "data/images"
annotation_dir = "data/annotations"

# Kiểm tra thư mục tồn tại
print("Thư mục images tồn tại:", os.path.exists(image_dir))
print("Thư mục annotation tồn tại:", os.path.exists(annotation_dir))

# Liệt kê các file trong thư mục
print("File trong images:", os.listdir(image_dir))
print("File trong annotation:", os.listdir(annotation_dir))

Thư mục images tồn tại: True
Thư mục annotation tồn tại: True
File trong images: ['receipt_00001.png', 'receipt_00002.png', 'receipt_00003.png', 'receipt_00004.png', 'receipt_00005.png', 'receipt_00006.png', 'receipt_00007.png', 'receipt_00008.png', 'receipt_00009.png', 'receipt_00010.png', 'receipt_00011.png', 'receipt_00012.png', 'receipt_00013.png', 'receipt_00014.png', 'receipt_00015.png', 'receipt_00016.png', 'receipt_00017.png', 'receipt_00018.png', 'receipt_00019.png', 'receipt_00020.png', 'receipt_00021.png', 'receipt_00022.png', 'receipt_00023.png', 'receipt_00024.png', 'receipt_00025.png', 'receipt_00027.png', 'receipt_00028.png', 'receipt_00029.png', 'receipt_00030.png', 'receipt_00031.png', 'receipt_00032.png', 'receipt_00033.png', 'receipt_00034.png', 'receipt_00035.png', 'receipt_00036.png', 'receipt_00037.png', 'receipt_00038.png', 'receipt_00039.png', 'receipt_00041.png', 'receipt_00042.png', 'receipt_00043.png', 'receipt_00044.png', 'receipt_00045.png', 'receipt_00046.

In [51]:
import os
import json
import shutil
from sklearn.model_selection import train_test_split

# Định nghĩa thư mục gốc và tỷ lệ chia dữ liệu
dataset_name = "dataset"
image_dir = "data/images"  # Thư mục chứa ảnh gốc
annotation_dir = "data/annotations"  # Thư mục chứa file JSON gốc
train_ratio, val_ratio, test_ratio = 0.8, 0.1, 0.1

# Tạo thư mục cho dataset
os.makedirs(dataset_name, exist_ok=True)
for split in ["train", "validation", "test"]:
    os.makedirs(os.path.join(dataset_name, split), exist_ok=True)

# Hàm xử lý file JSON để tạo ground truth
def process_cord_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    menu_items = []
    subtotal = None
    total_info = {}

    grouped_lines = {}
    for line in data["valid_line"]:
        group_id = line["group_id"]
        if group_id not in grouped_lines:
            grouped_lines[group_id] = []
        grouped_lines[group_id].append(line)

    for group_id, lines in grouped_lines.items():
        menu_item = {}
        for line in lines:
            category = line["category"]
            text = " ".join(word["text"] for word in line["words"])

 # Xử lý các danh mục menu
            if category.startswith("menu"):
                if category == "menu.nm":
                    menu_item["name"] = text
                elif category == "menu.cnt":
                    menu_item["count"] = text
                elif category == "menu.price":
                    menu_item["price"] = text
            elif category == "sub_total.subtotal_price":
                subtotal = text
            elif category.startswith("total"):
                if category == "total.total_price":
                    total_info["total_price"] = text
                elif category == "total.cashprice":
                    total_info["cash"] = text
                elif category == "total.changeprice":
                    total_info["change"] = text
                elif category == "total.menutype_cnt":
                    total_info["menu_types"] = text
                elif category == "total.menuqty_cnt":
                    total_info["menu_quantity"] = text

        if menu_item:
            menu_item.setdefault("count", None)
            menu_item.setdefault("price", None)
            menu_items.append(menu_item)

    ground_truth = {
        "menu": menu_items,
        "subtotal": subtotal,
        "total": total_info
    }
    return ground_truth

# Thu thập tất cả các file JSON và ảnh
data = []
for json_file in os.listdir(annotation_dir):
    if json_file.endswith(".json"):
        image_file = json_file.replace(".json", ".jpg")
        json_path = os.path.join(annotation_dir, json_file)
        image_path = os.path.join(image_dir, image_file)

        if os.path.exists(image_path):
            ground_truth = process_cord_json(json_path)
            data.append({
                "image_file": image_file,
                "image_path": image_path,
                "ground_truth": ground_truth
            })

# Chia dữ liệu thành train, validation, test
train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=test_ratio/(test_ratio + val_ratio), random_state=42)

# Hàm lưu dữ liệu vào thư mục và tạo metadata.jsonl
def save_split(split_data, split_name):
    split_dir = os.path.join(dataset_name, split_name)
    metadata_path = os.path.join(split_dir, "metadata.jsonl")

    with open(metadata_path, "w") as f:
        for item in split_data:
            # Sao chép ảnh vào thư mục tương ứng
            dest_image_path = os.path.join(split_dir, item["image_file"])
            shutil.copy(item["image_path"], dest_image_path)

            # Tạo dòng metadata
            metadata = {
                "file_name": item["image_file"],
                "ground_truth": json.dumps({"gt_parse": item["ground_truth"]})
            }
            f.write(json.dumps(metadata) + "\n")

# Lưu dữ liệu vào các thư mục
save_split(train_data, "train")
save_split(val_data, "validation")
save_split(test_data, "test")

print("Dataset đã được tạo xong!")

ValueError: With n_samples=0, test_size=None and train_size=0.8, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [53]:
import os
import json
import shutil
from sklearn.model_selection import train_test_split

# Định nghĩa thư mục gốc và tỷ lệ chia dữ liệu
dataset_name = "cord_dataset"
image_dir = "data/images"  # Thay bằng đường dẫn thực tế nếu cần
annotation_dir = "data/annotations"  # Thay bằng đường dẫn thực tế nếu cần
train_ratio, val_ratio, test_ratio = 0.8, 0.1, 0.1

# Kiểm tra thư mục
if not os.path.exists(image_dir):
    raise FileNotFoundError(f"Thư mục {image_dir} không tồn tại")
if not os.path.exists(annotation_dir):
    raise FileNotFoundError(f"Thư mục {annotation_dir} không tồn tại")

# Tạo thư mục cho dataset
os.makedirs(dataset_name, exist_ok=True)
for split in ["train", "validation", "test"]:
    os.makedirs(os.path.join(dataset_name, split), exist_ok=True)

# Hàm xử lý file JSON để tạo ground truth
def process_cord_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    menu_items = []
    subtotal = None
    total_info = {}

    grouped_lines = {}
    for line in data["valid_line"]:
        group_id = line["group_id"]
        if group_id not in grouped_lines:
            grouped_lines[group_id] = []
        grouped_lines[group_id].append(line)

    for group_id, lines in grouped_lines.items():
        menu_item = {}
        for line in lines:
            category = line["category"]
            text = " ".join(word["text"] for word in line["words"])

            if category.startswith("menu"):
                if category == "menu.nm":
                    menu_item["name"] = text
                elif category == "menu.cnt":
                    menu_item["count"] = text
                elif category == "menu.price":
                    menu_item["price"] = text
            elif category == "sub_total.subtotal_price":
                subtotal = text
            elif category.startswith("total"):
                if category == "total.total_price":
                    total_info["total_price"] = text
                elif category == "total.cashprice":
                    total_info["cash"] = text
                elif category == "total.changeprice":
                    total_info["change"] = text
                elif category == "total.menutype_cnt":
                    total_info["menu_types"] = text
                elif category == "total.menuqty_cnt":
                    total_info["menu_quantity"] = text

        if menu_item:
            menu_item.setdefault("count", None)
            menu_item.setdefault("price", None)
            menu_items.append(menu_item)

    ground_truth = {
        "menu": menu_items,
        "subtotal": subtotal,
        "total": total_info
    }
    return ground_truth

# Thu thập tất cả các file JSON và ảnh
data = []
for json_file in os.listdir(annotation_dir):
    if json_file.endswith(".json"):
        image_file_base = json_file.replace(".json", "")
        image_path = None
        for ext in [".jpg", ".png"]:  # Hỗ trợ nhiều định dạng ảnh
            temp_path = os.path.join(image_dir, image_file_base + ext)
            if os.path.exists(temp_path):
                image_path = temp_path
                image_file = image_file_base + ext
                break

        if image_path is None:
            print(f"Không tìm thấy ảnh cho {json_file}")
            continue

        json_path = os.path.join(annotation_dir, json_file)
        ground_truth = process_cord_json(json_path)
        data.append({
            "image_file": image_file,
            "image_path": image_path,
            "ground_truth": ground_truth
        })

# Kiểm tra xem có dữ liệu không
if not data:
    raise ValueError("Không tìm thấy dữ liệu nào để xử lý. Vui lòng kiểm tra thư mục images và annotation.")

# Chia dữ liệu thành train, validation, test
train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=test_ratio/(test_ratio + val_ratio), random_state=42)

# Hàm lưu dữ liệu vào thư mục và tạo metadata.jsonl
def save_split(split_data, split_name):
    split_dir = os.path.join(dataset_name, split_name)
    metadata_path = os.path.join(split_dir, "metadata.jsonl")

    with open(metadata_path, "w") as f:
        for item in split_data:
            dest_image_path = os.path.join(split_dir, item["image_file"])
            shutil.copy(item["image_path"], dest_image_path)

            metadata = {
                "file_name": item["image_file"],
                "ground_truth": json.dumps({"gt_parse": item["ground_truth"]})
            }
            f.write(json.dumps(metadata) + "\n")

# Lưu dữ liệu vào các thư mục
save_split(train_data, "train")
save_split(val_data, "validation")
save_split(test_data, "test")

print("Dataset đã được tạo xong!")

Dataset đã được tạo xong!
